<a href="https://colab.research.google.com/github/dunglucbac/Crawl-TIKI/blob/master/completing_proccessAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### EYEQ

In [ ]:
eyeq = {
    "message": "successful",
    "results": [
        {
            "box": {
                "xmin": 478,
                "ymin": 284,
                "xmax": 539,
                "ymax": 300
            },
            "value": "61A66666",
            "score": 0.9,
            "vehicle": {
                "score": 0.895,
                "type": "Car",
                "box": {
                    "xmin": 364,
                    "ymin": 166,
                    "xmax": 589,
                    "ymax": 328
                }
            },
            "model_make": {
                "make": "Mazda",
                "model": "3",
                "score": 0.5975329279899597
            },
            "color": [
                {
                    "color": "red",
                    "score": 0.78
                },
                {
                    "color": "white",
                    "score": 0.08
                },
                {
                    "color": "black",
                    "score": 0.04
                }
            ]
        },
        {
            "box": {
                "xmin": 122,
                "ymin": 289,
                "xmax": 184,
                "ymax": 304
            },
            "value": "48A55555",
            "score": 0.9,
            "vehicle": {
                "score": 0.909,
                "type": "Car",
                "box": {
                    "xmin": 70,
                    "ymin": 162,
                    "xmax": 307,
                    "ymax": 332
                }
            },
            "model_make": {
                "make": "Toyota",
                "model": "Corolla",
                "score": 0.7942538857460022
            },
            "color": [
                {
                    "color": "black",
                    "score": 0.65
                },
                {
                    "color": "white",
                    "score": 0.1
                },
                {
                    "color": "grey",
                    "score": 0.08
                }
            ]
        }
    ]
}

print([eyeq])

[{'message': 'successful', 'results': [{'box': {'xmin': 478, 'ymin': 284, 'xmax': 539, 'ymax': 300}, 'value': '61A66666', 'score': 0.9, 'vehicle': {'score': 0.895, 'type': 'Car', 'box': {'xmin': 364, 'ymin': 166, 'xmax': 589, 'ymax': 328}}, 'model_make': {'make': 'Mazda', 'model': '3', 'score': 0.5975329279899597}, 'color': [{'color': 'red', 'score': 0.78}, {'color': 'white', 'score': 0.08}, {'color': 'black', 'score': 0.04}]}, {'box': {'xmin': 122, 'ymin': 289, 'xmax': 184, 'ymax': 304}, 'value': '48A55555', 'score': 0.9, 'vehicle': {'score': 0.909, 'type': 'Car', 'box': {'xmin': 70, 'ymin': 162, 'xmax': 307, 'ymax': 332}}, 'model_make': {'make': 'Toyota', 'model': 'Corolla', 'score': 0.7942538857460022}, 'color': [{'color': 'black', 'score': 0.65}, {'color': 'white', 'score': 0.1}, {'color': 'grey', 'score': 0.08}]}]}]


In [ ]:
!pip install PyMySQL

     |████████████████████████████████| 51kB 2.8MB/s 


In [ ]:
import pandas as pd
import sqlite3
import pymysql.cursors

In [ ]:
def get_connection():
  connection = pymysql.connect(host='34.92.161.180',
                               user='ds_test',
                               password='5UjxfJN3BxmW8qhuUhGpwx4ZTamJhNYrvGX8mKFE',
                               db='ds_test',
                               charset='utf8mb4',                              
                               cursorclass=pymysql.cursors.DictCursor)
  return connection

In [ ]:
def process_data(eyeq, index):
  box = list(eyeq['results'][index]['box'].values())

  color = []
  color_score = []
  colors = [list(c.values()) for c in eyeq['results'][index]['color']]
  for c in colors:
    color.append(c[0])
    color_score.append(c[1])

  model_make = eyeq['results'][index]['model_make']['make']
  model = eyeq['results'][index]['model_make']['model']
  model_score = eyeq['results'][index]['model_make']['score']
  value = eyeq['results'][index]['value']
  vehicle_box = list(eyeq['results'][index]['vehicle']['box'].values())
  vehicle_score = eyeq['results'][index]['vehicle']['score']
  vehicle_type = eyeq['results'][index]['vehicle']['type']
  value_score = eyeq['results'][index]['score']

  return (box, value, value_score, vehicle_score, vehicle_type, vehicle_box, model_make, model, model_score, color, color_score)

In [ ]:
def insert_table(id_behind_image, message, box, value, value_score, vehicle_score, vehicle_type, vehicle_box, model_make, model, model_score, color, color_score, eyeq):
  query = """INSERT INTO request_history_eyeq (id_behind_image, message, lp_box, lp_value, lp_value_score, vehicle_type_score, vehicle_type, vehicle_box, vehicle_make, vehicle_model, vehicle_score, color, color_score, json) 
        VALUES ("{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}"
        )""".format(id_behind_image, message, box, value, value_score, vehicle_score, vehicle_type, vehicle_box, model_make, model, model_score, color, color_score, eyeq)
  try:
    cursor.execute(query)
    print('inserted succesfully')
  except Exception as err:
    print('ERROR BY INSERT:', err)

In [ ]:
def parse_data(eyeq):
  connect = get_connection()
  cursor = connect.cursor()

  message = eyeq['message']
  id_behind_image = "4409baab7836c3e18b476ab0e7ae34d98ad4db80.jpeg"
  length_result = len(eyeq['results'])
  
  for i in range(length_result):
    (box, value, value_score, vehicle_score, vehicle_type, vehicle_box, 
     model_make, model, model_score, color, color_score) = process_data(eyeq, i)
    insert_table(id_behind_image, message, box, value, value_score, vehicle_score, vehicle_type, vehicle_box, 
                 model_make, model, model_score, color, color_score, eyeq)

  connect.commit()

In [ ]:
parse_data(eyeq)

inserted succesfully
inserted succesfully


In [ ]:
pd.read_sql_query('SELECT * FROM request_history_eyeq', connect)

,id,id_behind_image,message,lp_box,lp_value,lp_value_score,vehicle_type_score,vehicle_type,vehicle_box,vehicle_make,vehicle_model,vehicle_score,color,color_score,json
0,2,4409baab7836c3e18b476ab0e7ae34d98ad4db80.jpeg,successful,"[122, 289, 184, 304]",48A55555,0.9,0.909,Car,"[70, 162, 307, 332]",Toyota,Corolla,0.7942538857460022,"['black', 'white', 'grey']","[0.65, 0.1, 0.08]","{'message': 'successful', 'results': [{'box': ..."
1,3,4409baab7836c3e18b476ab0e7ae34d98ad4db80.jpeg,successful,"[478, 284, 539, 300]",61A66666,0.9,0.895,Car,"[364, 166, 589, 328]",Mazda,3,0.5975329279899597,"['red', 'white', 'black']","[0.78, 0.08, 0.04]","{'message': 'successful', 'results': [{'box': ..."
2,4,4409baab7836c3e18b476ab0e7ae34d98ad4db80.jpeg,successful,"[122, 289, 184, 304]",48A55555,0.9,0.909,Car,"[70, 162, 307, 332]",Toyota,Corolla,0.7942538857460022,"['black', 'white', 'grey']","[0.65, 0.1, 0.08]","{'message': 'successful', 'results': [{'box': ..."


### GOOGLE VISION

In [ ]:
gv = {
  "textAnnotations": [
    {
      "locale": "und",
      "description": "29A\n613.37\n",
      "boundingPoly": {
        "vertices": [
          {
            "x": 13,
            "y": 5
          },
          {
            "x": 258,
            "y": 5
          },
          {
            "x": 258,
            "y": 193
          },
          {
            "x": 13,
            "y": 193
          }
        ]
      }
    },
    {
      "description": "29A",
      "boundingPoly": {
        "vertices": [
          {
            "x": 81,
            "y": 5
          },
          {
            "x": 231,
            "y": 7
          },
          {
            "x": 230,
            "y": 95
          },
          {
            "x": 80,
            "y": 93
          }
        ]
      }
    },
    {
      "description": "613.37",
      "boundingPoly": {
        "vertices": [
          {
            "x": 13,
            "y": 98
          },
          {
            "x": 258,
            "y": 99
          },
          {
            "x": 258,
            "y": 193
          },
          {
            "x": 13,
            "y": 192
          }
        ]
      }
    }
  ],
  "fullTextAnnotation": {
    "pages": [
      {
        "width": 280,
        "height": 200,
        "blocks": [
          {
            "boundingBox": {
              "vertices": [
                {
                  "x": 14,
                  "y": 4
                },
                {
                  "x": 259,
                  "y": 6
                },
                {
                  "x": 258,
                  "y": 194
                },
                {
                  "x": 13,
                  "y": 192
                }
              ]
            },
            "paragraphs": [
              {
                "boundingBox": {
                  "vertices": [
                    {
                      "x": 14,
                      "y": 4
                    },
                    {
                      "x": 259,
                      "y": 6
                    },
                    {
                      "x": 258,
                      "y": 194
                    },
                    {
                      "x": 13,
                      "y": 192
                    }
                  ]
                },
                "words": [
                  {
                    "boundingBox": {
                      "vertices": [
                        {
                          "x": 81,
                          "y": 5
                        },
                        {
                          "x": 231,
                          "y": 7
                        },
                        {
                          "x": 230,
                          "y": 95
                        },
                        {
                          "x": 80,
                          "y": 93
                        }
                      ]
                    },
                    "symbols": [
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 81,
                              "y": 12
                            },
                            {
                              "x": 121,
                              "y": 12
                            },
                            {
                              "x": 120,
                              "y": 92
                            },
                            {
                              "x": 80,
                              "y": 92
                            }
                          ]
                        },
                        "text": "2"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 129,
                              "y": 6
                            },
                            {
                              "x": 169,
                              "y": 6
                            },
                            {
                              "x": 168,
                              "y": 93
                            },
                            {
                              "x": 128,
                              "y": 93
                            }
                          ]
                        },
                        "text": "9"
                      },
                      {
                        "property": {
                          "detectedBreak": {
                            "type": "EOL_SURE_SPACE"
                          }
                        },
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 174,
                              "y": 6
                            },
                            {
                              "x": 231,
                              "y": 7
                            },
                            {
                              "x": 230,
                              "y": 95
                            },
                            {
                              "x": 173,
                              "y": 94
                            }
                          ]
                        },
                        "text": "A"
                      }
                    ]
                  },
                  {
                    "boundingBox": {
                      "vertices": [
                        {
                          "x": 13,
                          "y": 98
                        },
                        {
                          "x": 258,
                          "y": 99
                        },
                        {
                          "x": 258,
                          "y": 193
                        },
                        {
                          "x": 13,
                          "y": 192
                        }
                      ]
                    },
                    "symbols": [
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 13,
                              "y": 98
                            },
                            {
                              "x": 84,
                              "y": 98
                            },
                            {
                              "x": 84,
                              "y": 192
                            },
                            {
                              "x": 13,
                              "y": 192
                            }
                          ]
                        },
                        "text": "6"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 85,
                              "y": 98
                            },
                            {
                              "x": 115,
                              "y": 98
                            },
                            {
                              "x": 115,
                              "y": 192
                            },
                            {
                              "x": 85,
                              "y": 192
                            }
                          ]
                        },
                        "text": "1"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 116,
                              "y": 99
                            },
                            {
                              "x": 163,
                              "y": 99
                            },
                            {
                              "x": 163,
                              "y": 193
                            },
                            {
                              "x": 116,
                              "y": 193
                            }
                          ]
                        },
                        "text": "3"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 164,
                              "y": 99
                            },
                            {
                              "x": 194,
                              "y": 99
                            },
                            {
                              "x": 194,
                              "y": 193
                            },
                            {
                              "x": 164,
                              "y": 193
                            }
                          ]
                        },
                        "text": "."
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 195,
                              "y": 99
                            },
                            {
                              "x": 226,
                              "y": 99
                            },
                            {
                              "x": 226,
                              "y": 193
                            },
                            {
                              "x": 195,
                              "y": 193
                            }
                          ]
                        },
                        "text": "3"
                      },
                      {
                        "property": {
                          "detectedBreak": {
                            "type": "LINE_BREAK"
                          }
                        },
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 227,
                              "y": 99
                            },
                            {
                              "x": 258,
                              "y": 99
                            },
                            {
                              "x": 258,
                              "y": 193
                            },
                            {
                              "x": 227,
                              "y": 193
                            }
                          ]
                        },
                        "text": "7"
                      }
                    ]
                  }
                ]
              }
            ],
            "blockType": "TEXT"
          }
        ]
      }
    ],
    "text": "29A\n613.37\n"
  }
}

In [ ]:
conn = sqlite3.connect('gv.db')
cursor = conn.cursor()

In [ ]:
connection = get_connection()
cursor = connection.cursor()

In [ ]:
def createt_gv_table():
  query = """
      CREATE TABLE IF NOT EXISTS request_history_ggvision(
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          text_bounding TEXT,
          text_des TEXT,
          text_locale TEXT,
          json TEXT
      )
  """
  try:
    cursor.execute(query)
  except Exception as err:
    print('ERROR BY CREATE TABLE', err)
createt_gv_table()

ERROR BY CREATE TABLE (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'AUTOINCREMENT,\n          text_bounding TEXT,\n          text_des TEXT,\n          ' at line 2")


In [ ]:
def insert_gv(text_bounding, text_des, text_locale, gv):
  query = """ INSERT INTO request_history_ggvision (text_bounding, text_des, text_locale, json)
              VALUES ("{}", "{}", "{}", "{}")""".format(text_bounding, text_des, text_locale, gv)
  try:
    cursor.execute(query)
    print('inserted successfully')
  except Exception as err:
    print('ERROR BY INSERT:', err)

In [ ]:
def parse_gv(gv):
  text_bounding = gv['textAnnotations'][0]['boundingPoly']['vertices']
  text_des = gv['textAnnotations'][0]['description']
  text_locale = gv['textAnnotations'][0]['locale']

  insert_gv(text_bounding, text_des, text_locale, gv)

In [ ]:
parse_gv(gv)

inserted successfully


In [ ]:
def select_all():
  return cursor.execute('SELECT * FROM request_history_ggvision').fetchall()

In [ ]:
pd.read_sql_query('SELECT * FROM request_history_ggvision', connection)

,id,text_bounding,text_des,text_locale,json,time_stamp
0,1,"[{'x': 13, 'y': 5}, {'x': 258, 'y': 5}, {'x': ...",29A\n613.37\n,und,"{'textAnnotations': [{'locale': 'und', 'descri...",2020-07-15 08:47:36


In [ ]:
connection.commit()

### Car_image

In [ ]:
car_img = {
   "id_behind":"123.jpg",
   "id_front":"456.jpg",
   "license_plate_str":"51A1234",
   "license_plate_b64":"apoiwejfoawijef"
}

In [ ]:
!pip install pymysql

     |████████████████████████████████| 51kB 1.7MB/s 


In [ ]:
import pandas as pd
import sqlite3
import pymysql.cursors

In [ ]:
conn = sqlite3.connect('car_img.db')
cur = conn.cursor()

In [ ]:
id_behind = car_img['id_behind']
id_front = car_img['id_front']
lp_string = car_img['license_plate_str']
lp_b64 = car_img['license_plate_b64']

In [ ]:
query = """
  
"""